# Лабораторная работа 1. Прогнозирование моделью линейной регрессии

### 0. Загрузка данных

Перед нормированием и расчетом весов необходимо загрузить данные с помощью библиотеки numpy (```X``` - матрица признаков, ```y``` - матрица целевых значений).

In [49]:
import numpy as np

# получение данных из файлов
X = np.genfromtxt("./datasets/lab1_X.csv")
y = np.genfromtxt("./datasets/lab1_y.csv").reshape(50, 1)

print(f"Матрицы X и y:\n{X}\n{y}")


Матрицы X и y:
[[7.000e+00 1.590e+01 8.200e+00 5.100e+00 1.380e+01 2.290e+02 1.720e+02
  2.000e+01 2.104e+03]
 [7.200e+00 1.820e+01 7.400e+00 6.100e+00 1.430e+01 1.460e+02 1.670e+02
  2.910e+01 2.489e+03]
 [7.900e+00 1.970e+01 6.400e+00 4.700e+00 1.980e+01 1.740e+02 1.440e+02
  2.280e+01 2.428e+03]
 [7.700e+00 2.080e+01 6.900e+00 5.200e+00 1.710e+01 1.280e+02 1.110e+02
  4.270e+01 2.494e+03]
 [9.200e+00 1.590e+01 7.800e+00 5.300e+00 1.670e+01 1.690e+02 1.480e+02
  2.270e+01 2.094e+03]
 [7.600e+00 1.640e+01 6.700e+00 4.700e+00 1.550e+01 1.440e+02 1.500e+02
  2.790e+01 1.768e+03]
 [7.300e+00 1.830e+01 6.300e+00 4.900e+00 1.960e+01 1.380e+02 1.330e+02
  3.370e+01 1.982e+03]
 [7.900e+00 1.640e+01 6.800e+00 5.000e+00 1.760e+01 1.970e+02 1.550e+02
  2.660e+01 1.621e+03]
 [7.900e+00 1.700e+01 6.300e+00 4.400e+00 2.010e+01 1.820e+02 1.590e+02
  3.050e+01 1.631e+03]
 [8.000e+00 1.690e+01 8.200e+00 4.600e+00 1.550e+01 5.200e+02 1.970e+02
  1.910e+01 1.066e+03]
 [7.200e+00 1.760e+01 8.100e+00 5.2

Разделим все данные на выборку для обучения и выборку для валидации (```X_train```, ```y_train``` и ```X_test```, ```y_test``` соответственно)

In [65]:
# разделяем данные так: 70% для обучения, 30% для валидации
split = round(X.shape[0] * 0.7)

# Разделение для матрицы признаков
X_train = X[:split]
X_test = X[split:]

# Разделение для матрицы целевых значений
y_train = y[:split]
y_test = y[split:]

print(f"Размерности матриц: X_test - {X_test.shape}, X_train - {X_train.shape}, y_test - {y_test.shape}, y_train - {y_train.shape}")

Размерности матриц: X_test - (15, 9), X_train - (35, 9), y_test - (15, 1), y_train - (35, 1)


### 1. Нормирование (масштабирование) исходных данных.

Теперь необходимо нормализовать данные, поскольку разброс значенией данных слишком большой. Будет использоваться Z-нормализация

In [68]:
# функции для нахождения среднего и стандартного распределения
X_train = (X_train - np.mean(X_train)) / np.std(X_train)
X_test = (X_test - np.mean(X_test)) / np.std(X_test)

print(X_train)

[[-0.41047775 -0.39437582 -0.4083067  -0.41391524 -0.39817515 -0.00883409
  -0.11195881 -0.38695807  3.38342651]
 [-0.41011591 -0.39021464 -0.40975407 -0.41210603 -0.39727055 -0.15899816
  -0.12100484 -0.3704943   4.07997068]
 [-0.40884946 -0.38750084 -0.41156327 -0.41463892 -0.38731992 -0.1083404
  -0.16261657 -0.3818923   3.96960914]
 [-0.4092113  -0.38551071 -0.41065867 -0.41373432 -0.39220477 -0.19156386
  -0.22232036 -0.34588911  4.08901671]
 [-0.4064975  -0.39437582 -0.40903038 -0.4135534  -0.39292845 -0.11738643
  -0.15537975 -0.38207322  3.36533445]
 [-0.40939222 -0.39347121 -0.41102051 -0.41463892 -0.3950995  -0.16261657
  -0.15176134 -0.37266535  2.77553341]
 [-0.40993499 -0.39003372 -0.41174419 -0.41427708 -0.38768176 -0.17347181
  -0.18251784 -0.36217196  3.16270342]
 [-0.40884946 -0.39347121 -0.41083959 -0.41409616 -0.39130017 -0.06672867
  -0.14271531 -0.37501732  2.50958018]
 [-0.40884946 -0.39238569 -0.41174419 -0.41518168 -0.38677715 -0.09386676
  -0.13547849 -0.367961

### 2. Расчет весов линейной регрессии по аналитической формуле.

После нормализации можно рассчитать веса с помощью аналитической формулы (здесь также будут использоваться функции numpy)

In [70]:
cov_mat = np.matmul(np.transpose(X_train), X_train) # получение ковариационной матрицы
temp = np.matmul(np.linalg.inv(cov_mat), np.transpose(X_train)) # произведение обратной ковариационной матрицы и X транспонированной
w = np.matmul(temp, y_train) # получение матрицы весов

print(f"Полученные веса:\n{w}")

Полученные веса:
[[-105.34037223]
 [-458.9797658 ]
 [ 487.78769156]
 [-191.16376235]
 [ 110.10871375]
 [   2.60991367]
 [ -12.15662621]
 [  12.59801378]
 [   0.72561761]]


### 3. Построение и интепретация корреляционной матрицы. Определение степени мультиколлинеарности на основе числа обусловленности.

Теперь определим степень мультиколлинеарности на основе числа обусловленности. Будет использоваться функция ```eigvals``` из linalg для подсчета собственных значений.

In [67]:
print(f"Ковариационная матрица:\n{cov_mat}")
print(f"Число обусловленности: {np.linalg.cond(cov_mat)}")

Ковариационная матрица:
[[5.21230000e+03 7.05490000e+03 3.46229000e+03 1.99646000e+03
  9.15055000e+03 7.86938000e+04 7.51478000e+04 1.70223000e+04
  9.15624900e+05]
 [7.05490000e+03 1.15672200e+04 5.31096000e+03 3.27420000e+03
  1.36619100e+04 1.26726400e+05 1.18688600e+05 2.38341900e+04
  1.43612090e+06]
 [3.46229000e+03 5.31096000e+03 2.53680000e+03 1.53063000e+03
  6.49240000e+03 5.98916000e+04 5.62492000e+04 1.15256300e+04
  6.68457900e+05]
 [1.99646000e+03 3.27420000e+03 1.53063000e+03 9.56380000e+02
  3.89902000e+03 3.68230000e+04 3.47337000e+04 6.71442000e+03
  4.13999800e+05]
 [9.15055000e+03 1.36619100e+04 6.49240000e+03 3.89902000e+03
  1.72783100e+04 1.52031700e+05 1.45173600e+05 3.04579000e+04
  1.76337660e+06]
 [7.86938000e+04 1.26726400e+05 5.98916000e+04 3.68230000e+04
  1.52031700e+05 1.59805300e+06 1.39910700e+06 2.49649800e+05
  1.56747010e+07]
 [7.51478000e+04 1.18688600e+05 5.62492000e+04 3.47337000e+04
  1.45173600e+05 1.39910700e+06 1.33441000e+06 2.43579200e+05


### 4. Анализ регрессионных остатков.

Найдем среднеквадратическую ошибку (MSE) для матрицы ```X_train```, будем искать регрессионные остатки относительно ```y_train```.

In [76]:
# MSE для обучающий данных
Eps_train = np.matmul(X_train, w) - y_train
MSE_train = np.matmul(np.transpose(Eps_train), Eps_train) / X.shape[0]

print(f"MSE: {MSE_train[0][0]}")

MSE: 1.7560210016827977


Теперь найдем MSE для валидационной выборки (```X_test``` и ```y_test```).

In [77]:
# MSE для валидационных данных
Eps_test = np.matmul(X_test, w) - y_test
MSE_test = np.matmul(np.transpose(Eps_test), Eps_test) / X.shape[0]

print(f"MSE: {MSE_test[0][0]}")

MSE: 12.078294172537214
